In [ ]:
import numpy as np
import pandas as pd
from sympy import sieve
import colorcet as cc

import datashader as ds
import pandas as pd
from colorcet import fire
from datashader import transfer_functions as tf

import holoviews as hv
import geoviews as gv
from holoviews.operation.datashader import datashade
hv.extension('bokeh')

In [ ]:
cities = pd.read_csv("../input/cities.csv", index_col=['CityId'])
pnums = list(sieve.primerange(0, cities.shape[0]))
cities['isprime'] = cities.index.isin(pnums)

In [ ]:
cities.head()

In [ ]:
cvs = ds.Canvas(plot_width=500, plot_height=300)
points     = cvs.points(cities, 'X', 'Y')
all_cities = tf.shade(points, cmap=['lightgrey', 'black'], how='log')
all_cities 

# Sample first 100 cities 

In [ ]:
# https://ericphanson.com/blog/2016/the-traveling-salesman-and-10-lines-of-python/
import random, numpy, math, copy, matplotlib.pyplot as plt
cols = ['X', 'Y']
stadt = cities[cols].values.tolist()
tour = cities.index.values.tolist()

sample_cities = stadt[0:100]
sample_tour = tour[0:100]

xmin = min(pair[0] for pair in sample_cities)
xmax = max(pair[0] for pair in sample_cities)

ymin = min(pair[1] for pair in sample_cities)
ymax = max(pair[1] for pair in sample_cities)

def transform(pair):
    x = pair[0]
    y = pair[1]
    return [(x-xmin)*100/(xmax - xmin), (y-ymin)*100/(ymax - ymin)]

rescaled_cities = [ transform(b) for b in sample_cities] # rescale from 0 to 100

In [ ]:
%%time 

for temperature in numpy.logspace(0,5,num=100000)[::-1]:
    [i,j] = sorted(random.sample(range(len(sample_tour)),2));
    newTour =  sample_tour[:i] + sample_tour[j:j+1] + sample_tour[i+1:j] + sample_tour[i:i+1] + sample_tour[j+1:];
    
    oldDistances =  sum([ math.sqrt(sum([(sample_cities[sample_tour[(k+1) % len(sample_tour)]][d] - 
                                          sample_cities[sample_tour[k % len(sample_tour)]][d])**2 for d in [0,1] ])
                                   ) for k in [j,j-1,i,i-1]])
    newDistances =  sum([ math.sqrt(sum([(sample_cities[newTour[(k+1) % len(sample_tour)]][d] - 
                                          sample_cities[newTour[k % len(sample_tour)]][d])**2 for d in [0,1] ])
                                   ) for k in [j,j-1,i,i-1]])
    if math.exp( ( oldDistances - newDistances) / temperature) > random.random():
        sample_tour = copy.copy(newTour);
        
plt.plot([rescaled_cities[sample_tour[i % len(sample_tour)]][0] for i in range(len(sample_tour)+1)], 
         [rescaled_cities[sample_tour[i % len(sample_tour)]][1] for i in range(len(sample_tour)+1)], 'xb-');
plt.show()